In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import os
import pickle

In [3]:
# define path to data
__location__ = os.path.abspath("")
datapath = os.path.abspath(os.path.join(__location__, '..', 'data', 'gb_commons_embeedings_subset.feather'))

#read the feather file
df = pd.read_feather(datapath)
df.head()

,index,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,...,Speaker_MP,Speaker_Minister,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth,speech,Date_datetime
0,407774,ParlaMint-GB_2019-03-26-commons.u1,"Minutes of the House of Commons, Daily Session...",2019-03-26,Lower house,57,NaN,NaN,2019-03-26,NaN,...,MP,-,LAB,Labour,Opposition,"Powell, Lucy Maria",F,-,1. What steps he is taking to help ensure bett...,2019-03-26
1,407775,ParlaMint-GB_2019-03-26-commons.u2,"Minutes of the House of Commons, Daily Session...",2019-03-26,Lower house,57,NaN,NaN,2019-03-26,NaN,...,MP,-,CON,Conservative,None,"Hancock, Matthew John David",M,-,The Department of Health and Social Care works...,2019-03-26
2,407776,ParlaMint-GB_2019-03-26-commons.u3,"Minutes of the House of Commons, Daily Session...",2019-03-26,Lower house,57,NaN,NaN,2019-03-26,NaN,...,MP,-,LAB,Labour,Opposition,"Powell, Lucy Maria",F,-,I thank the Secretary of State very much for t...,2019-03-26
3,407777,ParlaMint-GB_2019-03-26-commons.u4,"Minutes of the House of Commons, Daily Session...",2019-03-26,Lower house,57,NaN,NaN,2019-03-26,NaN,...,MP,-,CON,Conservative,None,"Hancock, Matthew John David",M,-,I agree with the hon. Lady on both counts. My ...,2019-03-26
4,407778,ParlaMint-GB_2019-03-26-commons.u5,"Minutes of the House of Commons, Daily Session...",2019-03-26,Lower house,57,NaN,NaN,2019-03-26,NaN,...,MP,-,CON,Conservative,None,"Crouch, Tracey Elizabeth Anne",F,-,"On Friday, I met two clinical commissioning gr...",2019-03-26


In [4]:
# define pickle path
pklpath = os.path.abspath(os.path.join(__location__, '..', 'data', 'ParlaMint_GB_commons_embeddings_truncated.pkl'))

with (open(pklpath, "rb")) as openfile:
    while True:
        try:
            embeddings = pickle.load(openfile)
        except EOFError:
            break

In [6]:
vectors = np.array(list(embeddings.values()))
vkeys = np.array(list(embeddings.keys()))

In [7]:
#filter df to only include speeches with embeddings
# df = df[df['ID'].isin(vkeys)]
dfids = df['ID'].values
print(type(dfids))
print(len(df))
print(len(vkeys))

<class 'numpy.ndarray'>
41863
472782
